In [1]:
!python --version

Python 3.8.12


In [ ]:
! pip list

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.6/754.6 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 102.3 MB/s eta 0:00:00


  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert 0.2.3 requires transformers>=4.8.1, but you have transformers 3.0.0 which is incompatible.


  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-beno2xob
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-beno2xob


  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit e1f2f37055e7460d8427f6912579c0162cb69831
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached tokenizers-0.12.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.0rc4
    Uninstalling tokenizers-0.8.0rc4:
      Successfully uninstalled tokenizers-0.8.0rc4
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.0


    Uninstalling transformers-3.0.0:
      Successfully uninstalled transformers-3.0.0


In [2]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

/home/ubuntu/anaconda3/envs/python3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
import pandas as pd
train_data = pd.read_csv("data/train_data.csv")
test_data = pd.read_csv("data/test_data.csv")
submission = pd.read_csv("data/sample_submission.csv")

In [3]:
test_label =  [0 for i in range(len(test_data))]
test_data['label'] = test_label

In [4]:
del test_data['index']
test_data.head()

,title,label
0,유튜브 내달 2일까지 크리에이터 지원 공간 운영,0
1,어버이날 맑다가 흐려져…남부지방 옅은 황사,0
2,내년부터 국가RD 평가 때 논문건수는 반영 않는다,0
3,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,0
4,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,0


In [5]:
del train_data['index']
train_data.head()

,title,topic_idx
0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


In [6]:
print(len(train_data))
print(len(test_data))

45654
9131


In [7]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [8]:
#random seed 고정
np.random.seed(125)

# Training

In [10]:
bertmodel, vocab = get_pytorch_kobert_model()

/home/lab07/0707/dacon_topic_classificate/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/home/lab07/0707/dacon_topic_classificate/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train_data['topic_idx'])
train_data['topic_idx'] = encoder.transform(train_data['topic_idx'])
train_data.head()

# 매핑
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}

In [12]:
lst=[list(train_data.iloc[i]) for i in range(len(train_data))]
print(lst[:10])

[['인천→핀란드 항공기 결항…휴가철 여행객 분통', 4], ['실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화', 4], ['이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것', 4], ['NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합', 4], ['시진핑 트럼프에 중미 무역협상 조속 타결 희망', 4], ['팔레스타인 가자지구서 16세 소년 이스라엘군 총격에 사망', 4], ['인도 48년 만에 파키스탄 공습…테러 캠프 폭격종합2보', 4], ['美대선 TV토론 음담패설 만회실패 트럼프…사과 대신 빌클린턴 공격해 역효과', 4], ['푸틴 한반도 상황 진전 위한 방안 김정은 위원장과 논의', 4], ['특검 면죄부 받은 트럼프 스캔들 보도 언론 맹공…국민의 적', 4]]


In [13]:
#value error가 발생하여 list 형식으로 변경해주었다.

In [14]:
test_data = [list(test_data.iloc[i]) for i in range(len(test_data))]
print(test_data[:10])

[['유튜브 내달 2일까지 크리에이터 지원 공간 운영', 0], ['어버이날 맑다가 흐려져…남부지방 옅은 황사', 0], ['내년부터 국가RD 평가 때 논문건수는 반영 않는다', 0], ['김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것', 0], ['회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간', 0], ['야외서 생방송 하세요…액션캠 전용 요금제 잇따라', 0], ['월드컵 태극전사 16강 전초기지 레오강 입성종합', 0], ['미세먼지 속 출근길', 0], ['왓츠앱稅 230원에 성난 레바논 민심…총리사퇴로 이어져종합2보', 0], ['베트남 경제 고성장 지속…2분기 GDP 6.71% 성장', 0]]


In [15]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

using cached model. /home/lab07/0707/dacon_topic_classificate/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [16]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64  
warmup_ratio = 0.1
num_epochs = 5 
max_grad_norm = 1
log_interval = 20 
learning_rate = 5e-5

In [17]:
# Train / Valid set 분리
from sklearn.model_selection import train_test_split
train, valid = train_test_split(lst, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("valid shape is:", len(valid))

train shape is: 36523
valid shape is: 9131


In [18]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_valid = BERTDataset(valid, 0, 1, tok, max_len, True, False)

In [19]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)

/home/ubuntu/anaconda3/envs/python3/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
model = BERTClassifier(bertmodel, dr_rate=0.5).cuda()

In [22]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [23]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/home/ubuntu/anaconda3/envs/python3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [25]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [26]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [27]:
# 모델 학습 시작
for e in range(num_epochs):
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().cuda()
        segment_ids = segment_ids.long().cuda()
        valid_length= valid_length
        label = label.long().cuda()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval() # 평가 모드로 변경

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(valid_dataloader)):
        token_ids = token_ids.long().cuda()
        segment_ids = segment_ids.long().cuda()
        valid_length= valid_length
        label = label.long().cuda()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} valid acc {}".format(e+1, test_acc / (batch_id+1)))

 ====================== epoch 1 ======================


/tmp/ipykernel_11105/2973875419.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/571 [00:00<?, ?it/s]

Iteration   1 | Train Loss  2.0837 | Classifier Accuracy 0.06
Iteration  21 | Train Loss  1.9553 | Classifier Accuracy 0.14
Iteration  41 | Train Loss  1.9427 | Classifier Accuracy 0.16
Iteration  61 | Train Loss  1.7865 | Classifier Accuracy 0.21
Iteration  81 | Train Loss  1.4182 | Classifier Accuracy 0.30
Iteration 101 | Train Loss  1.1112 | Classifier Accuracy 0.39
Iteration 121 | Train Loss  0.8438 | Classifier Accuracy 0.46
Iteration 141 | Train Loss  0.6912 | Classifier Accuracy 0.51
Iteration 161 | Train Loss  0.6666 | Classifier Accuracy 0.55
Iteration 181 | Train Loss  0.5342 | Classifier Accuracy 0.58
Iteration 201 | Train Loss  0.5510 | Classifier Accuracy 0.61
Iteration 221 | Train Loss  0.4762 | Classifier Accuracy 0.63
Iteration 241 | Train Loss  0.4556 | Classifier Accuracy 0.65
Iteration 261 | Train Loss  0.3927 | Classifier Accuracy 0.67
Iteration 281 | Train Loss  0.3884 | Classifier Accuracy 0.68
Iteration 301 | Train Loss  0.3876 | Classifier Accuracy 0.69
Iteratio

/tmp/ipykernel_11105/2973875419.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(valid_dataloader)):


...

epoch 4 valid acc 0.8911230484631647
 ====================== epoch 5 ======================


  0%|          | 0/571 [00:00<?, ?it/s]

Iteration   1 | Train Loss  0.0769 | Classifier Accuracy 0.97
Iteration  21 | Train Loss  0.0862 | Classifier Accuracy 0.97
Iteration  41 | Train Loss  0.1711 | Classifier Accuracy 0.97
Iteration  61 | Train Loss  0.0349 | Classifier Accuracy 0.97
Iteration  81 | Train Loss  0.0834 | Classifier Accuracy 0.97
Iteration 101 | Train Loss  0.0222 | Classifier Accuracy 0.97
Iteration 121 | Train Loss  0.2436 | Classifier Accuracy 0.97
Iteration 141 | Train Loss  0.1069 | Classifier Accuracy 0.97
Iteration 161 | Train Loss  0.0539 | Classifier Accuracy 0.97
Iteration 181 | Train Loss  0.1284 | Classifier Accuracy 0.97
Iteration 201 | Train Loss  0.1621 | Classifier Accuracy 0.97
Iteration 221 | Train Loss  0.0397 | Classifier Accuracy 0.97
Iteration 241 | Train Loss  0.1451 | Classifier Accuracy 0.97
Iteration 261 | Train Loss  0.0467 | Classifier Accuracy 0.97
Iteration 281 | Train Loss  0.0295 | Classifier Accuracy 0.97
Iteration 301 | Train Loss  0.0541 | Classifier Accuracy 0.97
Iteratio

  0%|          | 0/143 [00:00<?, ?it/s]

epoch 5 valid acc 0.8896466905187836


# 추론

In [28]:
data_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [29]:
predictions =[]
with torch.no_grad():
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().cuda()
    segment_ids = segment_ids.long().cuda()
    valid_length= valid_length
    label = label.long().cuda()
    out = model(token_ids, valid_length, segment_ids)
    _, max_indices = torch.max(out, 1)

    predictions.extend(max_indices.squeeze(0).detach().cpu().numpy())

/tmp/ipykernel_11105/2741568512.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/143 [00:00<?, ?it/s]

In [30]:
print(predictions)

[2, 3, 2, 0, 3, 0, 5, 3, 4, 4, 4, 6, 4, 5, 6, 1, 6, 2, 4, 4, 4, 4, 4, 0, 0, 3, 6, 0, 5, 2, 3, 1, 4, 5, 2, 4, 5, 4, 6, 5, 5, 5, 5, 3, 0, 5, 3, 6, 1, 6, 0, 4, 6, 4, 5, 1, 4, 6, 3, 3, 4, 4, 4, 5, 3, 1, 5, 5, 2, 0, 3, 4, 3, 3, 0, 5, 1, 0, 1, 2, 6, 1, 5, 4, 6, 5, 4, 1, 6, 1, 5, 5, 4, 4, 4, 1, 6, 4, 1, 1, 5, 5, 4, 4, 5, 4, 3, 2, 0, 3, 2, 2, 2, 4, 3, 5, 3, 6, 2, 4, 5, 0, 6, 0, 2, 1, 5, 4, 0, 4, 0, 2, 6, 6, 4, 2, 6, 2, 1, 0, 5, 2, 3, 3, 4, 4, 1, 2, 3, 2, 6, 0, 1, 6, 4, 3, 5, 2, 0, 6, 1, 2, 3, 5, 5, 3, 6, 1, 1, 4, 1, 2, 2, 1, 3, 4, 2, 3, 6, 2, 5, 1, 5, 1, 5, 1, 2, 0, 3, 5, 2, 1, 0, 4, 6, 2, 6, 1, 3, 0, 1, 5, 4, 6, 5, 3, 5, 5, 5, 1, 1, 1, 3, 0, 1, 0, 4, 0, 6, 2, 4, 3, 5, 1, 0, 5, 0, 3, 4, 1, 6, 6, 3, 6, 4, 4, 5, 2, 2, 1, 5, 5, 1, 4, 0, 4, 5, 6, 6, 4, 4, 5, 1, 5, 5, 4, 5, 4, 2, 4, 5, 4, 0, 3, 3, 2, 0, 2, 5, 5, 4, 1, 0, 3, 6, 0, 3, 6, 5, 4, 6, 4, 4, 4, 4, 6, 4, 2, 3, 2, 2, 2, 5, 4, 5, 6, 4, 2, 2, 1, 6, 6, 0, 4, 6, 6, 3, 2, 1, 6, 5, 2, 5, 1, 0, 0, 3, 5, 0, 6, 5, 4, 1, 2, 2, 2, 1, 0, 4, 5, 0, 2, 4, 

In [31]:
submission['topic_idx']= predictions

In [33]:
submission

,index,topic_idx
0,45654,2
1,45655,3
2,45656,2
3,45657,0
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,3
9129,54783,2


In [34]:
submission.to_csv('classification_practice_0707.csv', index=False)